# Table 2.

Global diagnostics of in Hab1 and Hab2 simulations.

In [1]:
# Data analysis and viz libraries
import pandas as pd
import xarray as xr

In [2]:
# Local modules
import mypaths
from calc import (
    INTERPOLATOR,
    bond_albedo,
    cre_toa,
    greenhouse_effect,
    sfc_temp,
    spatial_mean,
    upper_atm_vap_mean,
)

# from load_thai import LOAD_CONF
from commons import MODELS
from names import names

## Choose cases

In [3]:
THAI_cases = ["Hab1", "Hab2"]

## Loading the data

Load the time-averaged data previously preprocessed.

In [4]:
# Load data
datasets = {}  # Create an empty dictionary to store all data
# for each of the THAI cases, create a nested directory for models
for THAI_case in THAI_cases:
    datasets[THAI_case] = {}
    for model_key in MODELS.keys():
        model_names = names[model_key]
        ds = xr.open_dataset(mypaths.datadir / model_key / f"{THAI_case}_time_mean_{model_key}.nc")
        if model_key == "LMDG":  # CHANGE TO ONLY LMDG!!!!!!!!!!
            lon_slice = slice(None, -1)  # omit the 180E longitude which is repeated
        else:
            lon_slice = slice(None, None)
        new_ds = {}
        for d in ds.data_vars:
            vrbl = ds[d]
            if model_names.x in ds[d].dims:
                vrbl = vrbl.isel(**{model_names.x: lon_slice})
            new_ds[d] = vrbl
        datasets[THAI_case][model_key] = xr.Dataset(new_ds)
    datasets[THAI_case]["ExoCAM"][names.exocam.lev].attrs.update(units="hPa")
    datasets[THAI_case]["ROCKE3D"][names.rocke3d.pres].attrs.update(units="hPa")
    datasets[THAI_case]["ROCKE3D"][names.rocke3d.lev].attrs.update(positive="up", units="1")

In [5]:
diags = {
    "t_sfc": {
        "func": lambda ds, model_key, const: sfc_temp(ds, model_key, const),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"T\textsubscript{s}",
    },
    "cre_sw": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="sw"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"CRE\textsubscript{SW}",
    },
    "cre_lw": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="lw"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"CRE\textsubscript{LW}",
    },
    "cre_tot": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="total"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"CRE",
    },
    "alb": {"func": bond_albedo, "fmt": lambda x: f"{x:.2f}", "title": r"$\alpha_\text{p}$"},
    "ghe_as": {
        "func": lambda ds, model_key, const: greenhouse_effect(
            ds, model_key, const, kind="all_sky"
        ),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"G\textsubscript{as}",
    },
    "ghe_cs": {
        "func": lambda ds, model_key, const: greenhouse_effect(
            ds, model_key, const, kind="clear_sky"
        ),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"G\textsubscript{cs}",
    },
    "vmr_1hpa": {
        "func": lambda ds, model_key, const: upper_atm_vap_mean(ds, model_key, const, [100]),
        "fmt": lambda x: fr"\num{{{x:.1e}}}",
        "title": r"VMR\textsubscript{\SI{1}{\hecto\pascal}}",
    },
}

In [6]:
results = {}
for THAI_case in THAI_cases:
    if THAI_case.endswith("1"):
        import const_ben1_hab1 as const
    else:
        import const_ben2_hab2 as const
    dfs = {}
    for model_key in MODELS.keys():
        _data = {}
        for varkey, vardict in diags.items():
            if varkey in ["t_sfc", "ghe_cs", "ghe_as", "vmr_1hpa"]:
                args = [const]
            else:
                args = []
            _data[vardict["title"]] = float(
                spatial_mean(
                    vardict["func"](datasets[THAI_case][model_key], model_key, *args),
                    names[model_key].x,
                    names[model_key].y,
                ).values
            )
        dfs[model_key] = pd.DataFrame(_data, index=pd.Index(name="GCM", data=[model_key]))
    results[THAI_case] = pd.concat(dfs, axis="index").droplevel(0).T
# df = pd.concat(results).T

In [7]:
for THAI_case in THAI_cases:
    df = results[THAI_case].T
    formatters = {k: {j["title"]: j["fmt"] for j in diags.values()}[k] for k in df.columns}
    print(fr"{{}} & \multicolumn{{{len(diags)}}}{{c}}{{{THAI_case}}} \\")
    print()
    print(
        df.to_latex(
            formatters=formatters,
            column_format="l" + "c" * len(diags),  # * len(THAI_cases)
            escape=False,
        ).replace(" " * 14, " ")
    )

{} & \multicolumn{8}{c}{Hab1} \\

\begin{tabular}{lcccccccc}
\toprule
{} & T\textsubscript{s} & CRE\textsubscript{SW} & CRE\textsubscript{LW} &   CRE & $\alpha_\text{p}$ & G\textsubscript{as} & G\textsubscript{cs} & VMR\textsubscript{\SI{1}{\hecto\pascal}} \\
GCM     &       &          &          &       &      &        &        &   \\
\midrule
ExoCAM  & 245.6 &    -37.9 &     15.5 & -22.4 & 0.24 &   10.3 &    5.8 &  \num{4.6e-07} \\
LMDG    & 242.2 &    -21.8 &      5.1 & -16.8 & 0.22 &    6.0 &    4.8 &  \num{5.8e-08} \\
ROCKE3D & 244.0 &    -44.6 &      9.3 & -35.3 & 0.27 &    9.3 &    6.7 &  \num{5.1e-07} \\
UM      & 231.6 &    -48.1 &     10.1 & -38.0 & 0.28 &    2.6 &   -0.0 &  \num{5.1e-07} \\
\bottomrule
\end{tabular}

{} & \multicolumn{8}{c}{Hab2} \\

\begin{tabular}{lcccccccc}
\toprule
{} & T\textsubscript{s} & CRE\textsubscript{SW} & CRE\textsubscript{LW} &   CRE & $\alpha_\text{p}$ & G\textsubscript{as} & G\textsubscript{cs} & VMR\textsubscript{\SI{1}{\hecto\pascal}} \\
GC